In [1]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
import cv2
from tqdm import tqdm
from keras import optimizers

from sklearn.cross_validation import KFold
from sklearn.metrics import fbeta_score
import time

Using Theano backend.
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
x_test = []

for i in range(40669):
    img = cv2.imread('../DB/test-jpg/test_{}.jpg'.format(i))
    x_test.append(cv2.resize(img, (64, 64)))
    
    
for i in range(20522):
    img = cv2.imread('../DB/test-jpg-additional/file_{}.jpg'.format(i))
    x_test.append(cv2.resize(img, (64, 64)))

In [11]:
label_map = {'agriculture': 14,
 'artisinal_mine': 5,
 'bare_ground': 1,
 'blooming': 3,
 'blow_down': 0,
 'clear': 10,
 'cloudy': 16,
 'conventional_mine': 2,
 'cultivation': 4,
 'habitation': 9,
 'haze': 6,
 'partly_cloudy': 13,
 'primary': 7,
 'road': 11,
 'selective_logging': 12,
 'slash_burn': 8,
 'water': 15}

inv_label_map = {value: key for key, value in label_map.items()}
print(inv_label_map)

{14: 'agriculture', 5: 'artisinal_mine', 1: 'bare_ground', 3: 'blooming', 0: 'blow_down', 10: 'clear', 16: 'cloudy', 2: 'conventional_mine', 4: 'cultivation', 9: 'habitation', 6: 'haze', 13: 'partly_cloudy', 7: 'primary', 11: 'road', 12: 'selective_logging', 8: 'slash_burn', 15: 'water'}


In [15]:
print(x_test[0])

[[[66 73 62]
  [67 74 67]
  [69 77 70]
  ..., 
  [81 94 88]
  [80 97 91]
  [73 78 70]]

 [[69 76 64]
  [70 77 68]
  [71 81 73]
  ..., 
  [77 88 80]
  [78 89 81]
  [73 80 72]]

 [[71 77 69]
  [69 79 69]
  [70 81 70]
  ..., 
  [73 82 70]
  [74 82 75]
  [74 85 76]]

 ..., 
 [[71 81 72]
  [69 75 65]
  [69 81 69]
  ..., 
  [72 81 70]
  [73 80 70]
  [75 83 74]]

 [[73 79 72]
  [66 75 67]
  [66 77 67]
  ..., 
  [71 80 69]
  [72 82 71]
  [73 85 77]]

 [[67 76 66]
  [70 78 71]
  [69 82 69]
  ..., 
  [71 78 70]
  [73 82 75]
  [75 84 78]]]


got 0.6 because I forgot to normalize x_test... :(

In [16]:
x_test  = np.array(x_test, np.float32)/255.

In [4]:
model = Sequential()
model.add(BatchNormalization(input_shape=(64, 64,3)))
model.add(Conv2D(32, kernel_size=(3, 3),padding='same', activation='relu'))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, kernel_size=(3, 3),padding='same', activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, kernel_size=(3, 3),padding='same', activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(256, kernel_size=(3, 3),padding='same', activation='relu'))
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(17, activation='sigmoid'))

model.load_weights('weights_kfold_1.h5')

In [5]:
def map_predictions(predictions, labels_map, thresholds):
    """
    Return the predictions mapped to their labels
    :param predictions: the predictions from the predict() method
    :param labels_map: the map
    :param thresholds: The threshold of each class to be considered as existing or not existing
    :return: the predictions list mapped to their labels
    """
    predictions_labels = []
    for prediction in predictions:
        labels = [labels_map[i] for i, value in enumerate(prediction) if value > thresholds[i]]
        predictions_labels.append(labels)

    return predictions_labels

In [18]:
predictions = model.predict(x_test)

In [19]:
thres = [0.07, 0.17, 0.2, 0.04, 0.23, 0.33, 0.24, 0.22, 0.1, 0.19, 0.23, 0.24, 0.12, 0.14, 0.25, 0.26, 0.16]
predicted_labels = map_predictions(predictions, inv_label_map, thres)

x_test_filename = ['test_{}.jpg'.format(i) for i in range(40669)]
x_test_filename2 = ['file_{}.jpg'.format(i) for i in range(20522)]
x_test_filename = np.hstack((x_test_filename, x_test_filename2))
print(x_test_filename.shape)

tags_list = [None] * len(predicted_labels)
for i, tags in enumerate(predicted_labels):
    tags_list[i] = ' '.join(map(str, tags))

final_data = [[filename.split(".")[0], tags] for filename, tags in zip(x_test_filename, tags_list)]

final_df = pd.DataFrame(final_data, columns=['image_name', 'tags'])
final_df.head()

final_df.to_csv('kele_xu.csv', index=False)

(61191,)
